In [1]:
import pandas as pd
import numpy as np


prune positions
iterate through positions update frontier

for position if there exists a higher value at same price remove
for position if there exists a lower price at same value

In [22]:
stuff = zip(np.random.choice(range(1, 10), 100), np.random.choice(range(1, 10), 100))

In [127]:
#price, value = zip(*stuff)
value = np.random.choice(range(1, 100), 10000)
price = [x/2 + y for x, y in zip(value, np.random.choice(range(1, 10), 10000))]

In [128]:
np.corrcoef(value, price)

array([[ 1.        ,  0.98371206],
       [ 0.98371206,  1.        ]])

In [123]:
def helper(val, d, up):
    if up:
        return max([v for k, v in d.iteritems() if k <= val])
    else:
        return min([v for k, v in d.iteritems() if k >= val])
        

def dominated(price, value):
    prices = np.unique(price)
    values = np.unique(value)
    best_vals = {p : max([value[i] for i, v in enumerate(price) if v == p]) for p in prices}
    best_pris = {v : min([price[i] for i, p in enumerate(value) if p == v]) for v in values}
    bests = [(k,v) for k, v in best_vals.iteritems()]+[(v,k) for k, v in best_pris.iteritems()]
    bests_ = [(p,v) for p, v in bests if (v >= helper(p, best_vals, True)) and (p <= helper(v, best_pris, False))]
    return set(bests_)

In [129]:
dominated(price, value)

{(1, 1),
 (2, 3),
 (3, 5),
 (4, 7),
 (5, 9),
 (6, 11),
 (7, 13),
 (8, 15),
 (9, 17),
 (10, 19),
 (11, 21),
 (12, 23),
 (13, 25),
 (14, 27),
 (15, 29),
 (16, 31),
 (17, 33),
 (18, 35),
 (19, 37),
 (20, 39),
 (21, 41),
 (22, 43),
 (23, 45),
 (24, 47),
 (25, 49),
 (26, 51),
 (27, 53),
 (28, 55),
 (29, 57),
 (30, 59),
 (31, 61),
 (32, 63),
 (33, 65),
 (34, 67),
 (35, 69),
 (36, 71),
 (37, 73),
 (38, 75),
 (39, 77),
 (40, 79),
 (41, 81),
 (42, 83),
 (43, 85),
 (44, 87),
 (45, 89),
 (46, 91),
 (47, 93),
 (48, 95),
 (49, 97),
 (50, 99)}